In [13]:
import os
import json
import pandas as pd
import traceback

In [14]:
from langchain_openai import ChatOpenAI

In [26]:
from dotenv import load_dotenv
load_dotenv("/Users/ss11/mcqgen/src/.env")

True

In [24]:
KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
llm=ChatOpenAI(api_key=KEY, model="gpt-3.5-turbo", temperature=0.5)